In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))


import logging.config
from dobermann.config import settings

logging.config.dictConfig(settings.LOGGING)

In [2]:
from dobermann import indicators
from dobermann.btest import Strategy, Candle, PositionType


class BollingerTestStrategy(Strategy):

    def __init__(self):
        super().__init__()

        self.ind_bollinger = indicators.BollingerBands()

    def on_candle(self, candle: Candle):
        # Get current values of indicator
        lower_band, sma, upper_band = self.ind_bollinger.calculate(candle)
        if sma is None:
            # Индикатор еще не прогрет, ждем следующей свечи
            return

        price = candle.close

        if not self.exchange.active_position and price < lower_band:
            self.exchange.open_market_position(PositionType.LONG)

        elif self.exchange.active_position and price > upper_band:
            self.exchange.close_market_position()


In [3]:
from dobermann.btest import backtest, Timeframe
import datetime as dt

ticker = 'BTCUSDT'
start = dt.datetime(2022, 1, 1)
end = dt.datetime(2022, 4, 1)
timeframe = Timeframe.H1

test_report = await BollingerTestStrategy().backtest(ticker, timeframe, start, end)

INFO    2022-07-10 16:45:22,515 dobermann.btest: Fetching candles data... 
2161it [00:02, 783.59it/s]
INFO    2022-07-10 16:45:25,292 dobermann.btest: Perform strategy... 
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2161/2161 [00:07<00:00, 297.80it/s]
INFO    2022-07-10 16:45:32,551 dobermann.btest: Done! 


In [4]:
test_report.equity_graph

alt.LayerChart(...)

In [5]:
test_report.summary

{'mean_profit_ratio': Decimal('1.00430603'),
 'gmean_profit_ratio': Decimal('1.00319138'),
 'max_dropdown': Decimal('0.89029840'),
 'total_trades': 27,
 'success_trades': 19,
 'fail_trades': 8,
 'success_trades_ratio': Decimal('0.70370370')}